In [1]:
%pip install git+https://github.com/haccht/scrapli_ipython
%reload_ext scrapli_ipython

  Cloning https://github.com/haccht/scrapli_ipython to /tmp/pip-req-build-6_jfkoma
  Running command git clone --filter=blob:none --quiet https://github.com/haccht/scrapli_ipython /tmp/pip-req-build-6_jfkoma
  Resolved https://github.com/haccht/scrapli_ipython to commit 1689bdeaa7126a06c2dc95beb17f9ce8e2539728
  Preparing metadata (setup.py) ... done
  Using cached scrapli-2022.7.30.post1-py3-none-any.whl (138 kB)
  Using cached ntc_templates-3.1.0-py3-none-any.whl (332 kB)
  Using cached textfsm-1.1.3-py2.py3-none-any.whl (44 kB)
  Using cached ssh2_python-1.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.5 MB)
  Using cached future-0.18.2-py3-none-any.whl
  Created wheel for scrapli-ipython: filename=scrapli_ipython-0.0.1-py3-none-any.whl size=3419 sha256=efb3b95f7e176a881995575b659419a8fae6ee69cea25c2960f21fd6504b0cfe
  Stored in directory: /tmp/pip-ephem-wheel-cache-lf5dk8b4/wheels/03/4e/5e/bdaf8b4c75fd5ab6c7e7b6a3c094141eb505378915586f6921
Successfully built scra

You can establish a connection to a device with `%scrapli` line magic.

In [2]:
%scrapli --platform cisco_iosxr 192.168.121.105

Username: vagrant
Password: ········


`%%cmd` cell magic allows you to execute all commands in the cell.

In [3]:
%%cmd
show interface brief
show ipv4 interface brief

show interface brief
Fri Nov 11 20:21:10.709 UTC

               Intf       Intf        LineP              Encap  MTU        BW
               Name       State       State               Type (byte)    (Kbps)
--------------------------------------------------------------------------------
                Nu0          up          up               Null  1500          0
       Mg0/0/CPU0/0          up          up               ARPA  1514          0
          Gi0/0/0/0  admin-down  admin-down               ARPA  1514    1000000
          Gi0/0/0/1  admin-down  admin-down               ARPA  1514    1000000
          Gi0/0/0/2  admin-down  admin-down               ARPA  1514    1000000
-- 
show ipv4 interface brief
Fri Nov 11 20:21:10.909 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
MgmtEth0/0/CPU0/0              192.168.121.105 Up              Up       default
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default
GigabitEthe

`%%cmd` cell magic accepts a jinja2 template and execute the rendered commands.

In [4]:
%%cmd resp
{% for i in range(3) %}
show interface Gi0/0/0/{{i}}
{% endfor %}

show interface Gi0/0/0/0
Fri Nov 11 20:21:11.109 UTC
GigabitEthernet0/0/0/0 is administratively down, line protocol is administratively down
  Interface state transitions: 6
  Hardware is GigabitEthernet, address is 5254.00dd.7f8c (bia 5254.00dd.7f8c)
  Internet address is Unknown
  MTU 1514 bytes, BW 1000000 Kbit (Max: 1000000 Kbit)
     reliability 255/255, txload 0/255, rxload 0/255
  Encapsulation ARPA,
  Full-duplex, 1000Mb/s, unknown, link type is force-up
  output flow control is off, input flow control is off
  Carrier delay (up) is 10 msec
  loopback not set,
  Last link flapped 00:00:41
  Last input 00:00:51, output never
  Last clearing of "show interface" counters never
  5 minute input rate 0 bits/sec, 0 packets/sec
  5 minute output rate 0 bits/sec, 0 packets/sec
     35297 packets input, 2117820 bytes, 0 total input drops
     0 drops for unrecognized upper-level protocol
     Received 0 broadcast packets, 35297 multicast packets
              0 runts, 0 giants, 0 thrott

`%%cmd` cell magic can also capture command results as a scrapli MultiReponse, so that you can manupulate them.

In [5]:
resp[0].textfsm_parse_output()

[{'interface': 'GigabitEthernet0/0/0/0',
  'link_status': 'administratively down',
  'admin_state': 'administratively down',
  'hardware_type': 'GigabitEthernet',
  'address': '5254.00dd.7f8c',
  'bia': '5254.00dd.7f8c',
  'description': '',
  'ip_address': 'Unknown',
  'mtu': '1514',
  'duplex': 'Full',
  'speed': '1000Mb/s',
  'bandwidth': '1000000 Kbit',
  'encapsulation': 'ARPA'}]

`%%configure` cell magic loads cell contents into the configuration.

In [6]:
%%configure
{% for i in range(3) %}
no interface Gi0/0/0/{{i}} shutdown
{% endfor %}

show configuration
commit

no interface Gi0/0/0/0 shutdown

no interface Gi0/0/0/1 shutdown

no interface Gi0/0/0/2 shutdown

show configuration
Fri Nov 11 20:21:13.439 UTC
Building configuration...
!! IOS XR Configuration 5.3.3
interface GigabitEthernet0/0/0/0
 no shutdown
!
interface GigabitEthernet0/0/0/1
 no shutdown
!
interface GigabitEthernet0/0/0/2
 no shutdown
!
end
commit
Fri Nov 11 20:21:13.649 UTC



Let's validate whether all ports were successfully opened.

In [7]:
%%cmd resp
{% for i in range(3) %}
show interface Gi0/0/0/{{i}}
{% endfor %}

show interface Gi0/0/0/0
Fri Nov 11 20:21:14.169 UTC
GigabitEthernet0/0/0/0 is up, line protocol is up
  Interface state transitions: 7
  Hardware is GigabitEthernet, address is 5254.00dd.7f8c (bia 5254.00dd.7f8c)
  Internet address is Unknown
  MTU 1514 bytes, BW 1000000 Kbit (Max: 1000000 Kbit)
     reliability 255/255, txload 0/255, rxload 0/255
  Encapsulation ARPA,
  Full-duplex, 1000Mb/s, unknown, link type is force-up
  output flow control is off, input flow control is off
  Carrier delay (up) is 10 msec
  loopback not set,
  Last link flapped 00:00:01
  Last input 00:00:00, output never
  Last clearing of "show interface" counters never
  5 minute input rate 0 bits/sec, 0 packets/sec
  5 minute output rate 0 bits/sec, 0 packets/sec
     35302 packets input, 2118120 bytes, 0 total input drops
     0 drops for unrecognized upper-level protocol
     Received 0 broadcast packets, 35302 multicast packets
              0 runts, 0 giants, 0 throttles, 0 parity
     0 input errors, 0 C

In [8]:
for i in range(3):
    assert 'administratively down' not in resp[i].result, f"Gi0/0/0/{i} should not be shutdown"

In [9]:
%close